# PPO
---

### 1. Import the Necessary Packages

In [1]:
%matplotlib inline
import wandb

from model.ppo_2 import PPO
import numpy as np
from env.wrappers import LunarContinuous, LunarLanderWithUnknownWind,LunarLanderWithKnownWind
from utils.logger import WandbSummaryWritter

### 2. Instantiate the Model

Setup the hyperparameters in the code cell below.

In [2]:
misc_hyperparameters = {
    'save_freq': 0 ,  
    'val_freq': 10,
    'val_iter': 10,
    'env': LunarLanderWithKnownWind
}

Initialise wandb session in the code cell below.

In [ ]:
MAX_TOTAL_TIMESTEPS_TO_TRAIN = 500
VAL_ITER = 30
MAX_RUN_COUNT = 30
sweep_config = {
    'method': 'bayes', 
    'metric': {
        'name': 'validation_rewards',
        'goal': 'maximize'
    },
    'parameters': {
        'actor_lr': {
            "distribution": "uniform",
            "min": 1e-5,
            "max": 0.1
        },
        'critic_lr': {
            "distribution": "uniform",
            "min": 1e-5,
            "max": 0.1
        },
        'adp_lr': {
            "distribution": "uniform",
            "min": 1e-5,
            "max": 0.1
        },
        'gamma': {
            'min': 0.9,
            'max': 1.
        },
        'lam': {
            'min': 0.9,
            'max': 1.
        },
        'max_grad_norm': {
            "min": 0.1,
            "max": 10.0, 
        },
        'n_updates_per_iteration': {
            'values': list(range(1, 21))
        },
        'num_envs': {
            'values': list(range(1, 100))
        },
        'anneal_lr': {
            'values': [True, False]
        },
        'num_steps': {
            'distribution': 'q_uniform',
            'min': 300,
            'max': 4000,
            'q': 100
        },
        'adp_num_steps': {
            'distribution': 'q_uniform',
            'min': 200,
            'max': 1000,
            'q': 10
        },
        'anneal_discount': {
            'distribution': 'q_uniform',
            'min': 1,
            'max': 1000,
            'q': 10
        },
        'batches': {
            'distribution': 'q_uniform',
            "min": 1,     # 2^0
            "max": 1024,  # 2^10
            "q": 2 
        }
    }
}

In [4]:
def train_model(config = None):
    logger = WandbSummaryWritter(project='lunar', config =config)
    ppo = ppo = PPO(logger,**misc_hyperparameters) if config is None else PPO(summary_writter=logger, **config, **misc_hyperparameters)
    ppo.train()

    val_rews, val_dur = ppo.validate(VAL_ITER, True, True)

    wandb.log({
        "validation_rewards": val_rews,
        "validation_duration": val_dur,
        "maximum_validation_reward": np.argmax(val_rews),
        "max_reward_video": wandb.Video(f"./videos/rl-video{np.argmax(val_rews)}-episode-{np.argmax(val_rews)}.mp4", fps=4, format="mp4")
    })

    # logger["delta_t"] = str(round(logger['delta_t'], 2))

Initialise the model fo the desired timestamps. Alternatively can specify a checkpoint to continue training.

In [5]:
sweep_id = wandb.sweep(sweep_config, project="lunar")
wandb.agent(sweep_id, function=train_model, count=MAX_RUN_COUNT)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Create sweep with ID: q44u0ys7
Sweep URL: https://wandb.ai/pmsaraiva2712-tum/lunar/sweeps/q44u0ys7


wandb: Agent Starting Run: xd92oei9 with config:
wandb: 	actor_lr: 0.07041309019796743
wandb: 	adp_lr: 0.0496149131147736
wandb: 	adp_num_steps: 260
wandb: 	anneal_discount: 280
wandb: 	anneal_lr: False
wandb: 	batches: 70
wandb: 	critic_lr: 0.03513488285322935
wandb: 	gamma: 0.986623171458458
wandb: 	lam: 0.9022184049586168
wandb: 	max_grad_norm: 3.4527110463690436
wandb: 	n_updates_per_iteration: 3
wandb: 	num_envs: 94
wandb: 	num_steps: 3100
wandb: Currently logged in as: mohamedrostom (pmsaraiva2712-tum). Use `wandb login --relogin` to force relogin
wandb: WARNING Ignored wandb.init() arg project when running a sweep.



-------------------- Iteration #1 --------------------
Average Episodic Return: -325.08
Average Actor Loss: -0.21142
Average Critic Loss: 1562.754366830679
Average KL Divergence: 0.012544951658212365
Iteration took: 3.01 secs, of which rollout took 2.47 secs and gradient updates took 0.53 secs
Current actor learning rate: 0.005
Current critic learning rate: 0.0075
------------------------------------------------------


-------------------- Iteration #2 --------------------
Average Episodic Return: -231.57
Average Actor Loss: -0.21272
Average Critic Loss: 1070.3282539661113
Average KL Divergence: 0.010671996380831562
Iteration took: 2.98 secs, of which rollout took 2.44 secs and gradient updates took 0.53 secs
Current actor learning rate: 0.005
Current critic learning rate: 0.0075
------------------------------------------------------


-------------------- Iteration #1 --------------------
Average adp Loss: 29.38668
Iteration took: 2.48 secs, of which rollout took 2.23 secs and gradi

/home/mohamedrostom62/ADLR/tum-adlr-ws25-16/.venv/lib/python3.11/site-packages/gym/wrappers/record_video.py:75: UserWarning: WARN: Overwriting existing videos at /home/mohamedrostom62/ADLR/tum-adlr-ws25-16/videos folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


...

MoviePy - Building video /home/mohamedrostom62/ADLR/tum-adlr-ws25-16/videos/rl-video0-episode-0.mp4.
MoviePy - Writing video /home/mohamedrostom62/ADLR/tum-adlr-ws25-16/videos/rl-video0-episode-0.mp4



wandb: WARNING `fps` argument does not affect the frame rate of the video when providing a file path or raw bytes.


MoviePy - Done !
MoviePy - video ready /home/mohamedrostom62/ADLR/tum-adlr-ws25-16/videos/rl-video0-episode-0.mp4


actor_learning_rate,▁▁
adp_learning_rate,▁
average_actor_loss,█▁
average_adapt_loss,▁
average_critic_loss,█▁
average_episode_rewards,▁█
critic_learning_rate,▁▁
maximum_validation_reward,▁
simulated_iterations,▁█▁
validation_duration,▁
validation_rewards,▁


wandb: Ctrl + C detected. Stopping sweep.


In [ ]:
api = wandb.Api()

# Fetch the sweep object
sweep = api.sweep(f"pmsaraiva2712-tum/lunar/{sweep_id}")

# Fetch all runs from the sweep
runs = sweep.runs

# Sort runs by the metric you are optimizing for, e.g., 'val_loss'
best_run = sorted(runs, key=lambda run: run.summary.get('val_rewards', float('-inf')), reverse=True)[0]

# Extract best hyperparameters and metrics
best_params = best_run.config
best_metrics = best_run.summary

# Print the best hyperparameters and metrics
print("Best Hyperparameters:", best_params)
print("Best Metrics:", best_metrics)

Best Hyperparameters: {'lam': 0.9821062547914114, 'gamma': 0.913815261726064, 'adp_lr': 0.08626140546523067, 'batches': 516, 'actor_lr': 0.034561042911770035, 'num_envs': 59, 'anneal_lr': True, 'critic_lr': 0.002892075552063189, 'num_steps': 2000, 'adp_num_steps': 910, 'max_grad_norm': 1.084814579512531, 'anneal_discount': 50, 'n_updates_per_iteration': 20}
Best Metrics: {'_runtime': 2934.173429823, '_step': 303, '_timestamp': 1734325500.5551257, '_wandb': {'runtime': 2934}, 'actor_learning_rate': 0.005, 'adp_learning_rate': 0.005, 'average_actor_loss': -0.21090720078392097, 'average_adapt_loss': 4.623626652312884, 'average_critic_loss': 64.72636713956297, 'average_episode_rewards': 186.6697540283203, 'critic_learning_rate': 0.0075, 'iteration_compute': '12.23', 'simulated_iterations': 84, 'val_durs': 901, 'val_rews': 155.49781812688917}
